# CrewAI: Orchestrating Collaborative Agent Systems

**Objective:** This notebook provides a hands-on guide to CrewAI, a framework designed for engineering sophisticated multi-agent systems. You will learn how to create a "crew" of autonomous AI agents that collaborate to achieve complex goals.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Philosophy:** CrewAI's strength lies in its intuitive, high-level abstractions for defining agents with specific roles, assigning them tasks, and defining the workflow (process) they should follow. It simplifies the creation of systems where multiple agents work together, each contributing its specialized skills.

## 1. Setup

First, we'll install the necessary libraries. CrewAI uses LangChain components under the hood. We will also install the `tavily-python` library directly to create a custom search tool, which helps avoid complex dependency issues.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

missing = []
for key in ["OPENAI_API_KEY", "TAVILY_API_KEY"]:
    if not os.getenv(key):
        missing.append(key)

if missing:
    print(f"ERROR: Missing required environment variables: {', '.join(missing)}. Please check your .env file.")
else:
    print("All required environment variables are present.")

All required environment variables are present.


In [4]:
# Dependency & version verification
import importlib

modules = [
    ("crewai", "crewai"),
    ("langchain_openai", "langchain-openai"),
    ("dotenv", "python-dotenv"),
    ("tavily", "tavily-python"),
]

for mod, pip_name in modules:
    try:
        m = importlib.import_module(mod)
        version = getattr(m, "__version__", "<no __version__ attr>")
        print(f"OK: {mod} (pip: {pip_name}) version {version}")
    except Exception as e:
        print(f"FAIL: Could not import {mod} (pip: {pip_name}) -> {e}")

# Quick functional smoke checks
try:
    from crewai import Agent, Task, Crew, Process
    print("CrewAI core classes imported successfully.")
except Exception as e:
    print(f"CrewAI import failed: {e}")

try:
    from langchain_openai import ChatOpenAI
    test_llm = ChatOpenAI(model="gpt-4o", temperature=0)
    print("LangChain OpenAI client instantiated (will require valid OPENAI_API_KEY for real calls).")
except Exception as e:
    print(f"LangChain OpenAI client failed: {e}")

try:
    from tavily import TavilyClient
    if os.getenv("TAVILY_API_KEY"):
        TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        print("Tavily client instantiated successfully.")
    else:
        print("Skipping Tavily client instantiation: TAVILY_API_KEY missing.")
except Exception as e:
    print(f"Tavily client failed: {e}")

OK: crewai (pip: crewai) version 1.2.1
OK: langchain_openai (pip: langchain-openai) version <no __version__ attr>
OK: dotenv (pip: python-dotenv) version <no __version__ attr>
OK: tavily (pip: tavily-python) version <no __version__ attr>
CrewAI core classes imported successfully.
LangChain OpenAI client instantiated (will require valid OPENAI_API_KEY for real calls).
Tavily client instantiated successfully.


## 2. Foundational Crew: The Two-Agent Team

Our first example will be a simple, two-agent crew designed to plan a trip. This will introduce the three core concepts of CrewAI:

-   **`Agent`**: A role-playing entity with a specific `role`, `goal`, and `backstory`.
-   **`Task`**: A specific unit of work to be performed by an agent.
-   **`Crew`**: A collection of agents and tasks, along with a defined `process` for execution.

In [5]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Use a LangChain model as the LLM for the agents
llm = ChatOpenAI(model="gpt-4o")

# Agent 1: The Travel Expert
travel_agent = Agent(
    role='Expert Travel Agent',
    goal='Create a detailed, 3-day itinerary for a trip.',
    backstory='You have 20 years of experience in luxury travel planning and know all the hidden gems.',
    verbose=True,
    llm=llm
)

# Agent 2: The Local Food Critic
food_critic = Agent(
    role='Local Food Critic',
    goal='Recommend the best, most authentic local restaurants for the trip.',
    backstory='You are a famous food blogger who lives in the destination city and is an expert on its culinary scene.',
    verbose=True,
    llm=llm
)

# Task 1: Plan the Itinerary (for the Travel Agent)
plan_itinerary = Task(
    description='Create a 3-day travel itinerary for a trip to Tokyo, Japan. Focus on cultural sites and activities.',
    expected_output='A markdown file with a day-by-day plan, including morning, afternoon, and evening activities.',
    agent=travel_agent
)

# Task 2: Recommend Restaurants (for the Food Critic)
# This task uses the output of the first task as its context.
recommend_restaurants = Task(
    description='Based on the planned itinerary, recommend one authentic restaurant for each day of the trip.',
    expected_output='A list of 3 restaurant names, each with a brief description and why it fits the itinerary.',
    agent=food_critic,
    context=[plan_itinerary]
)

# Form the crew with a sequential process
trip_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary, recommend_restaurants],
    process=Process.sequential,
    verbose=True
)

print("--- Kicking off the Trip Planning Crew ---")
trip_result = trip_crew.kickoff()

print("\n--- Final Itinerary ---")
print(trip_result)

--- Kicking off the Trip Planning Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3b3aeff6-ed6d-4e75-8dee-c3be95c5059d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Task: Create a 3-day travel itinerary for a trip to Tokyo, Japan. Focus on cultural sites and activities.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

C:\Users\labadmin\AppData\Roaming\Python\Python311\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  # 3-Day Tokyo Cultural Itinerary                                                                               │
│                                                                                                                 │
│  ## Day 1: Traditional Tokyo                                                                                    │
│                                                                                                                 │
│  ### Morning                                                                                                    │
│  - **Visit Asakusa and Senso-ji Temple**: Begin your day in the historic district of Asakusa. Explore           │
│  Senso-ji, Tokyo's oldest and most significant temple. Don't miss the vibrant Nakamise Street leading up to     │
│  the temple, lined with traditional shops and street food stalls.                                               │
│                                                                                                                 │
│  ### Afternoon                                                                                                  │
│  - **Lunch at Kaminarimon Street**: Enjoy local delights such as tempura or sushi in one of the cozy            │
│  restaurants around Asakusa.                                                                                    │
│  - **Ueno Park and Tokyo National Museum**: Head to Ueno Park, a large public park that's perfect for a         │
│  leisurely stroll. Visit the Tokyo National Museum to explore an extensive collection of Japanese art and       │
│  antiquities.                                                                                                   │
│                                                                                                                 │
│  ### Evening                                                                                                    │
│  - **Dinner at a Traditional Izakaya**: Experience a traditional Japanese pub in the Ueno area. Try a variety   │
│  of Japanese dishes in a lively atmosphere.                                                                     │
│  - **Tokyo Skytree**: Conclude your day with a visit to the Tokyo Skytree, getting breathtaking views of the    │
│  city illuminated at night.                                                                                     │
│                                                                                                                 │
│  ## Day 2: Modern Meets Tradition                                                                               │
│                                                                                                                 │
│  ### Morning                                                                                                    │
│  - **Meiji Shrine Visit**: Start your day at the serene Meiji Shrine, located next to the fashionable Harajuku  │
│  area. Enjoy a peaceful walk through the forested path leading to the shrine.                                   │
│                                                                                                                 │
│  ### Afternoon                                         

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Day 1: Komagata Dozeu (浅草駒形どぜう)**                                                                  │
│     - **Description**: Located near Asakusa, Komagata Dozeu offers a traditional Edo-style dining experience,   │
│  specializing in dozeu (loach fish). The historical atmosphere and authentic dishes make it an ideal choice     │
│  following a day of exploring Tokyo's historic sites.                                                           │
│     - **Why it fits the itinerary**: After visiting Senso-ji Temple and Ueno Park, Komagata Dozeu provides a    │
│  continuation of historical immersion with its traditional Japanese ambiance and cuisine. Perfect for           │
│  experiencing the culture of Edo-period Tokyo.                                                                  │
│                                                                                                                 │
│  2. **Day 2: Omotesando Ukaitei (表参道 うかい亭)**                                                             │
│     - **Description**: A luxurious, teppanyaki-style restaurant located in Omotesando, offering a fusion of     │
│  modern and traditional Japanese dishes with an emphasis on presentation and quality ingredients.               │
│     - **Why it fits the itinerary**: Situated in a fashionable district, Omotesando Ukaitei aligns with the     │
│  blend of modern and traditional elements explored during Day 2. It's a great stop after shopping in Harajuku   │
│  and Omotesando, providing a sophisticated edge before the lively night at Golden Gai.                          │
│                                                                                                                 │
│  3. **Day 3: Inokashira Park's Harmonica Yokocho Alley (吉祥寺 ハーモニカ横丁)**                                │
│     - **Description**: This area is adjacent to Inokashira Park and is known for its bohemian flair and         │
│  variety of eateries and small bars, each offering distinctive local flavors. It's a lively alley that retains  │
│  a rustic charm.                                                                                                │
│     - **Why it fits the itinerary**: Following a morning at the Ghibli Museum and tranquility of Inokashira     │
│  Park, dining at Harmonica Yokocho provides a perfect blend of local culture and cuisine. It captures the       │
│  artistic and relaxed vibe of Kichijoji before heading to a fine dining experience in Roppongi Hills.           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Itinerary ---
1. **Day 1: Komagata Dozeu (浅草駒形どぜう)**  
   - **Description**: Located near Asakusa, Komagata Dozeu offers a traditional Edo-style dining experience, specializing in dozeu (loach fish). The historical atmosphere and authentic dishes make it an ideal choice following a day of exploring Tokyo's historic sites.
   - **Why it fits the itinerary**: After visiting Senso-ji Temple and Ueno Park, Komagata Dozeu provides a continuation of historical immersion with its traditional Japanese ambiance and cuisine. Perfect for experiencing the culture of Edo-period Tokyo.

2. **Day 2: Omotesando Ukaitei (表参道 うかい亭)**  
   - **Description**: A luxurious, teppanyaki-style restaurant located in Omotesando, offering a fusion of modern and traditional Japanese dishes with an emphasis on presentation and quality ingredients.
   - **Why it fits the itinerary**: Situated in a fashionable district, Omotesando Ukaitei aligns with the blend of modern and traditional elements explored du

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fe6663d4-47bc-41ab-9bac-39ce3bdbc8a4                                                                     │
│  Agent: Local Food Critic                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3b3aeff6-ed6d-4e75-8dee-c3be95c5059d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. **Day 1: Komagata Dozeu (浅草駒形どぜう)**                                                    │
│     - **Description**: Located near Asakusa, Komagata Dozeu offers a traditional Edo-style dining experience,   │
│  specializing in dozeu (loach fish). The historical atmosphere and authentic dishes make it an ideal choice     │
│  following a day of exploring Tokyo's historic sites.                                                           │
│     - **Why it fits the itinerary**: After visiting Senso-ji Temple and Ueno Park, Komagata Dozeu provides a    │
│  continuation of historical immersion with its traditional Japanese ambiance and cuisine. Perfect for           │
│  experiencing the culture of Edo-period Tokyo.                                                                  │
│                                                                                                                 │
│  2. **Day 2: Omotesando Ukaitei (表参道 うかい亭)**                                                             │
│     - **Description**: A luxurious, teppanyaki-style restaurant located in Omotesando, offering a fusion of     │
│  modern and traditional Japanese dishes with an emphasis on presentation and quality ingredients.               │
│     - **Why it fits the itinerary**: Situated in a fashionable district, Omotesando Ukaitei aligns with the     │
│  blend of modern and traditional elements explored during Day 2. It's a great stop after shopping in Harajuku   │
│  and Omotesando, providing a sophisticated edge before the lively night at Golden Gai.                          │
│                                                                                                                 │
│  3. **Day 3: Inokashira Park's Harmonica Yokocho Alley (吉祥寺 ハーモニカ横丁)**                                │
│     - **Description**: This area is adjacent to Inokashira Park and is known for its bohemian flair and         │
│  variety of eateries and small bars, each offering distinctive local flavors. It's a lively alley that retains  │
│  a rustic charm.                                                                                                │
│     - **Why it fits the itinerary**: Following a morning at the Ghibli Museum and tranquility of Inokashira     │
│  Park, dining at Harmonica Yokocho provides a perfect blend of local culture and cuisine. It captures the       │
│  artistic and relaxed vibe of Kichijoji before heading to a fine dining experience in Roppongi Hills.           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are avai

## 3. Adding Tools to a CrewAI Agent

CrewAI agents can use tools to access external information and capabilities. Instead of relying on pre-built wrappers which can cause dependency issues, we can create our own custom tool by wrapping the Tavily Python client. This is a robust and flexible approach.

In [6]:
from crewai.tools import BaseTool
from tavily import TavilyClient

# 1. Create a custom tool by inheriting from BaseTool
class TavilySearchTool(BaseTool):
    name: str = "TavilySearch"
    description: str = "A tool that can be used to search the web with Tavily for up-to-date information."
    
    def _run(self, query: str) -> str:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        response = client.search(query=query, search_depth="basic")
        return response['results']

# Instantiate our custom tool
search_tool = TavilySearchTool()

# 2. Create an agent and assign the custom tool to it
market_researcher = Agent(
    role='Market Research Analyst',
    goal='Find and summarize the latest news about a specific company.',
    backstory='You are a skilled analyst who can quickly find and synthesize financial news.',
    tools=[search_tool], # Assign the custom tool here
    verbose=True,
    llm=llm
)

# 3. Create a task for the agent
research_company_task = Task(
    description='Find the top 3 news headlines for NVIDIA since September 2, 2025, summarize them and provide the news date',
    expected_output='A numbered list of 3 headlines, each followed by a one-sentence summary.',
    agent=market_researcher
)

# 4. Form a single-agent crew to run the task
research_crew = Crew(
    agents=[market_researcher],
    tasks=[research_company_task],
    verbose=True
)

print("--- Kicking off the Research Crew with Tools ---")
research_result = research_crew.kickoff()

print("\n--- Final Research Summary ---")
print(research_result)

--- Kicking off the Research Crew with Tools ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 273ece58-e250-4a18-b091-fa63c6a57322                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Find the top 3 news headlines for NVIDIA since September 2, 2025, summarize them and provide the news    │
│  date                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'url': 'https://www.youtube.com/watch?v=n0CIMNb770c', 'title': "I'm buying as much Nvidia stock as I can     │
│  before they announce this.", 'content': 'Dan Ives - "I\'m buying as much Nvidia stock as I can before they     │
│  announce this."\n\nThe NVIDIA Report\n65 likes\n3810 views\n2 Sep 2025\n.\nnvidia stock today\nnvidia stock    │
│  price prediction\nnvidia stock predictions\nnvidia stock crash\nnvidia stock news\nnvidia stock                │
│  analysis\n\n“We Post Daily Nvidia Videos, So By Not Subscribing To The Channel Your Missing All Nvidia Latest  │
│  News”.\n.✨ 👇 SUBSCRIBEHERE https://www.youtube.com/channel/UCKjflY8gRC1ojOrs20s_YTg?sub_confirmation=1\n     │
│  \n\nnvidia,jim cramer nvidia analysis,gpu technology,jim cramer nvidia insights,nvidia ai advancements,jim     │
│  cramer,nvidia\'s future,stocks to buy or sell,nvidia stock predictions for 2025,nvidia stock news,nvidia       │
│  earnings report analysis,nvidia stock analysis,nvidia latest news,nvidia blackwell chip updates,how to pick    │
│  stocks,best stocks,nvidia stock price,nvidia stock surge,best stocks to buy now,nvda earnings,cramer\'s        │
│  nvidia stock prediction,nvidia ai stock\n** All video and pictures are used with permission or in accordance   │
│  with the copyright owner\'s stated policies and use allowance, or applicable fair use laws. **                 │
│  \n\n\n\n\nKeywords\n#NVIDIA #NVIDIAStock #NVDA #AITech #AIStocks #TechStocks #StockMarket #Investingnvidia     │
│  stock,how to pick stocks,nvidiastockanalysis,stockstobuynow,finance stock,nvda stock,nvda stock analysis,ai    │
│  stocks,best ai stocks,nvidia news,nvidia stock price,nvda stock news,nvidia earnings,nvda earnings,is nvidia   │
│  stock a buy,stock market,nvidia stock analysis,tech stocks,nvidia stock predictions,why NVIDIA is              │
│  exploding,NVIDIA stock to the moon,NVIDIA price target,NVIDIA stock crash,NVIDIA 10x potential,should I buy    │
│  NVIDIA\nnvidia stock\nnvda stock\nnvidia stock news\nnvidia stock prediction\nnvda stock analysis\nnvidia      │
│  stock forecast 2025\nnvda earnings\nnvidia stock split\nnvidia stock update\nshould i buy nvidia sto...        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. "I'm buying as much Nvidia stock as I can before they announce this." - This video suggests increased       │
│  enthusiasm for NVIDIA stock due to expected announcements, as expressed by Dan Ives, indicating investor       │
│  interest and speculative anticipation about NVIDIA's future prospects. (September 2, 2025)                     │
│  2. "NVIDIA Announces Financial Results for Second Quarter Fiscal 2026" - NVIDIA reported its quarterly cash    │
│  dividend and provided financial performance updates, showcasing stability and shareholder engagement by        │
│  announcing dividends payable on October 2, 2025.                                                               │
│  3. "NVIDIA Sales Jump 56%, a Sign the A.I. Boom Isn't Slowing Down" - NVIDIA's significant sales increase      │
│  reflects the accelerating momentum of the AI industry, driving the company to become the world's first $4      │
│  trillion company, underlining its leadership in the AI space.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6cf41c11-b925-4098-9d85-bbd5ce37e20d                                                                     │
│  Agent: Market Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 273ece58-e250-4a18-b091-fa63c6a57322                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. "I'm buying as much Nvidia stock as I can before they announce this." - This video suggests   │
│  increased enthusiasm for NVIDIA stock due to expected announcements, as expressed by Dan Ives, indicating      │
│  investor interest and speculative anticipation about NVIDIA's future prospects. (September 2, 2025)            │
│  2. "NVIDIA Announces Financial Results for Second Quarter Fiscal 2026" - NVIDIA reported its quarterly cash    │
│  dividend and provided financial performance updates, showcasing stability and shareholder engagement by        │
│  announcing dividends payable on October 2, 2025.                                                               │
│  3. "NVIDIA Sales Jump 56%, a Sign the A.I. Boom Isn't Slowing Down" - NVIDIA's significant sales increase      │
│  reflects the accelerating momentum of the AI industry, driving the company to become the world's first $4      │
│  trillion company, underlining its leadership in the AI space.                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Research Summary ---
1. "I'm buying as much Nvidia stock as I can before they announce this." - This video suggests increased enthusiasm for NVIDIA stock due to expected announcements, as expressed by Dan Ives, indicating investor interest and speculative anticipation about NVIDIA's future prospects. (September 2, 2025)
2. "NVIDIA Announces Financial Results for Second Quarter Fiscal 2026" - NVIDIA reported its quarterly cash dividend and provided financial performance updates, showcasing stability and shareholder engagement by announcing dividends payable on October 2, 2025. 
3. "NVIDIA Sales Jump 56%, a Sign the A.I. Boom Isn't Slowing Down" - NVIDIA's significant sales increase reflects the accelerating momentum of the AI industry, driving the company to become the world's first $4 trillion company, underlining its leadership in the AI space.


## 4. Advanced Capability: Hierarchical Process

For more complex workflows, CrewAI offers a `hierarchical` process. In this mode, a manager agent is dynamically nominated to orchestrate the crew, delegating tasks to other agents and synthesizing the final result. This is suitable for problems that require more dynamic coordination.

In [7]:
# Ensure Task, Crew, and Process are available (in case earlier cells haven't been run)
from crewai import Task, Crew, Process

# Re-using the agents from our first example

# The tasks are the same, but we don't need to specify context
plan_itinerary_h = Task(
    description='Create a 3-day travel itinerary for a trip to Rome, Italy. Focus on historical sites.',
    expected_output='A markdown file with a day-by-day plan.',
    agent=travel_agent
)

recommend_restaurants_h = Task(
    description='Recommend one authentic restaurant for each day of the trip to Rome.',
    expected_output='A list of 3 restaurant names with descriptions.',
    agent=food_critic
)

# Form the crew with a hierarchical process
hierarchical_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary_h, recommend_restaurants_h],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=llm # Specify an LLM for the manager agent
)

print("--- Kicking off the Hierarchical Crew ---")
hierarchical_result = hierarchical_crew.kickoff()

print("\n--- Final Hierarchical Result ---")
print(hierarchical_result)

--- Kicking off the Hierarchical Crew ---


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Task: What are the must-see historical sites in Rome, Italy for a 3-day itinerary, and what would be an ideal  │
│  sequence for visiting them to maximize historical learning and enjoyment?                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **3-Day Historical Itinerary for Rome, Italy**                                                                 │
│                                                                                                                 │
│  **Day 1: Ancient Rome Unveiled**                                                                               │
│                                                                                                                 │
│  - **Morning: Colosseum and Roman Forum**                                                                       │
│    - Start your journey at the grand Colosseum, the iconic symbol of Rome’s enduring history. Book a            │
│  skip-the-line private guided tour to dive deep into tales of gladiators and epic battles.                      │
│    - Following the Colosseum, transition to the Roman Forum, the epicenter of ancient Roman political life.     │
│  Explore the ruins where Julius Caesar once walked, including the Senate House and the Temple of Saturn.        │
│                                                                                                                 │
│  - **Lunch: Al Gladiatore**                                                                                     │
│    - A charming traditional restaurant near the Colosseum. Enjoy authentic Roman specialties such as Cacio e    │
│  Pepe.                                                                                                          │
│                                                                                                                 │
│  - **Afternoon: Palatine Hill**                                                                                 │
│    - Head to the serene Palatine Hill, where, according to legend, Romulus founded Rome. Discover the ruins of  │
│  imperial palaces and enjoy panoramic views of the city.                                                        │
│                                                                                                                 │
│  - **Evening: Trastevere**                                                                                      │
│    - Wander through the vibrant and historic district of Trastevere. Have dinner in a quaint trattoria and      │
│  soak in the bohemian vibes.                                                                                    │
│                                                                                                                 │
│  **Day 2: Rome's Religious and Artistic Heritage**                                                              │
│                                                                                                                 │
│  - **Morning: Vatican City**                                                                                    │
│    - Begin with a guided tour of the Vatican Museums. Marvel at masterpieces from antiquity to the              │
│  Renaissance, culminating in Michelangelo's Sistine Chapel ceiling.                                             │
│    - Continue to St. Peter’s Basilica; climb to the top of the dome for an astonishing view of the Vatican and  │
│  Rome.                                                                                                          │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Recommend one authentic restaurant for each day of the trip to Rome, based on the provided itinerary.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  For your 3-day historical itinerary in Rome, here are my recommendations for one authentic restaurant to       │
│  visit each day:                                                                                                │
│                                                                                                                 │
│  **Day 1: Ancient Rome and Trastevere**                                                                         │
│  After exploring the Colosseum and Roman Forum, head over to Trastevere, a charming neighborhood known for its  │
│  authentic Roman cuisine. I recommend dining at **Trattoria Da Enzo al 29**. This is a beloved spot among       │
│  locals and offers classic Roman dishes like carbonara and saltimbocca in a cozy setting. Make sure to reserve  │
│  a table in advance, as it gets quite busy!                                                                     │
│                                                                                                                 │
│  **Day 2: Vatican City and Surroundings**                                                                       │
│  After a day of exploring the Vatican Museums, St. Peter's Basilica, and the Sistine Chapel, unwind with a      │
│  meal at **Ristorante Arlù**. Located near the Vatican, this family-run restaurant offers authentic Roman       │
│  cuisine with a twist. The friendliness of the staff and the quality of dishes like the cacio e pepe and osso   │
│  buco will make it a memorable dining experience.                                                               │
│                                                                                                                 │
│  **Day 3: Baroque Rome and Via Veneto**                                                                         │
│  Conclude your historical exploration with a visit to the magnificent Baroque sites such as the Pantheon and    │
│  Capitoline Hill. For dinner near Via Veneto, I recommend **Girarrosto Fiorentino**, which somewhat defies its  │
│  name by offering Italian cuisine with an emphasis on perfectly grilled meats. The atmosphere is elegant yet    │
│  welcoming making it a great spot to reflect on your Roman adventures while savoring authentic dishes.          │
│                                                                                                                 │
│  Enjoy your culinary and historical journey through Rome!                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Hierarchical Result ---
For your 3-day historical itinerary in Rome, here are my recommendations for one authentic restaurant to visit each day:

**Day 1: Ancient Rome and Trastevere**  
After exploring the Colosseum and Roman Forum, head over to Trastevere, a charming neighborhood known for its authentic Roman cuisine. I recommend dining at **Trattoria Da Enzo al 29**. This is a beloved spot among locals and offers classic Roman dishes like carbonara and saltimbocca in a cozy setting. Make sure to reserve a table in advance, as it gets quite busy!

**Day 2: Vatican City and Surroundings**  
After a day of exploring the Vatican Museums, St. Peter's Basilica, and the Sistine Chapel, unwind with a meal at **Ristorante Arlù**. Located near the Vatican, this family-run restaurant offers authentic Roman cuisine with a twist. The friendliness of the staff and the quality of dishes like the cacio e pepe and osso buco will make it a memorable dining experience.

**Day 3: Baroque Rom

## Lab Conclusion

In this lab, you've learned the fundamentals of CrewAI. You've seen how to define agents with distinct roles, create tasks for them to perform, and assemble them into a crew that can work together sequentially or be managed hierarchically.

**Key Takeaways:**
- CrewAI is excellent for problems that can be broken down into distinct roles and responsibilities.
- The `Agent`, `Task`, and `Crew` abstractions provide a clear and intuitive way to structure multi-agent systems.
- Creating custom tools by inheriting from `BaseTool` is a reliable way to add capabilities and avoid dependency conflicts.
- The `hierarchical` process allows for more dynamic, manager-led coordination for complex, non-linear tasks.